# Add Features from SDE into Feature Service

The script utilizes both Esri's Destkop Python library 'arcpy' & the web Python API.  Connect to ArcGIS Online or Enterprise
parse through various datapoints and add them into a service.

In [1]:
# Import modules
import json
import arcpy
import getpass
from arcpy import env
from arcgis.gis import *

In [2]:
# Funtion to delete temporary files
def delete_json(temp_file, value):
    if value == 0:
        if os.path.exists(temp_file):
            try:
                os.remove(temp_file)
            except OSError as e:  ## if failed, report it back to the user ##
                print("Error: %s - %s." % (e.filename, e.strerror))
    else:
        try:
            if arcpy.Exists(temp_file):
                arcpy.Delete_management(temp_file)
            else:
                pass
        except (arcpy.ExecuteError, arcpy.ExecuteWarning) as e:
            print("Error: %s" % e)

In [3]:
# Yield successive n-sized chunks from l.
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

## Connect to ArcGIS Online - Python API
Estbalish connection, search for content by name to return the item's ID.  Once found, connect to relevant index dataset, and truncate the service in preparation for data load.

In [4]:
# Utilize getpass for hiding password for demonstration purposes
password = getpass.getpass()

········


In [5]:
# Establish connection to a WebGIS organization
# ArcGIS Online
gis = GIS('http://www.arcgis.com', 'abrown_citygov', password)

# For ArcGIS Enterprise, connect as "https:<server DNS>/<web adaptor for portal>"
# Example: GIS('http://neenterprise.esri.com/portal', 'username', password)  

In [6]:
# Search by title
search = gis.content.search(query="title: PhillyCrimeExample", item_type="Feature Service")
search

[<Item title:"PhillyCrimeExample" type:Feature Layer Collection owner:abrown_citygov>]

In [7]:
# From search list, return the index for service one is interseted in connecting to
item = search[0]
itemID = item.id
itemID

'1d1ee09e66084db0a189ee0e86ba9689'

In [8]:
# Establish connection to Feature Layer item
feature_layer_item = gis.content.get(itemID)
flayers = feature_layer_item.layers
flayer = flayers[0]
flayer

<FeatureLayer url:"https://services6.arcgis.com/0p6i4J6xhQas4Unf/arcgis/rest/services/PhillyCrimeExample/FeatureServer/0">

In [25]:
# Truncate the dataset, cannot be utlized on layers with synch
flayer.manager.truncate()

# Alternative Option, utilize delete_features method in class FeatureLayer
# flayer.delete_features(where="1=1")

# Return total number of records after insert
total_result = flayer.query(where='1=1', out_fields='*')
print(len(total_result))

0


## Connect to Enterprise Geodatabase

Establish connection to enterprise geodatabase.  List feature classes, locate dataset of interest. Convert dataset into a json file. Loop through JSON file to pull out features and add to a service.

In [26]:
# Establish arcpy desktop workspace/scratch workspaces
wspace = r'C:\Users\alex8694\AppData\Roaming\ESRI\Desktop10.6\ArcCatalog\Enterprise_SDE.sde'
env.workspace = wspace
env.scratchWorkspace = r"D:\Scratch\Scratch.gdb"

# Create placeholder for temporary json file
temp_json = r"D:\jsonfeatures1.json"
delete_json(temp_json, 0)

In [27]:
# List Feature Classes in Enterprise Geodatabase
fclist = arcpy.ListFeatureClasses()

In [28]:
# Locate dataset of interest
for item in fclist:
    if item == 'Enterprise.SDE.PhillyCrimeSubset1':
        # Create temporary in memory layer of dataset
        arcpy.MakeFeatureLayer_management(item, "layer")
        
        # Convert to temporary json file
        arcpy.FeaturesToJSON_conversion("layer", temp_json)
        
        # Delete temporary memory layer
        arcpy.Delete_management("layer")
    else:
        pass

## Prepare data for insert into Feature Service

Manipulate the data coming from arcpy insert cursor, be aware of your time zone properties, add to a service.

In [23]:
# Open json file, read in features section
file = open(temp_json)
content = json.loads(file.read())
new_features = (content["features"])

# Utilize the chunks generator to create groups of 4000 features
# Be aware of your data type.  Complex geometry polygons, the maximum you might be able to chunk is 500-1000,
# Starting to approach the limits of add, look at Appending or Overwriting your service content.
new_items = chunks(new_features, 4000)

# Loop through each grouping of data, add to service.
for item in new_items:
    print(len(item))
    try:
        flayer.edit_features(adds=item, updates=None, deletes=None, gdb_version=None, use_global_ids=False,
                             rollback_on_failure=True)
    except ConnectionResetError as e:
        print(e)
        # Might want to add additional loops or trys if failed.
        pass
    except ConnectionError as e:
        print(e)
        pass
        # Might want to add additional loops or trys if failed.
    
file.close()

1873


In [24]:
# Return total number of records after insert
query_result = flayer.query(where='1=1', out_fields='*')
print(len(query_result))

# Delete temporary file
delete_json(temp_json, 0)

1873
